# Pydelmod example Notebook to post-processed DSM2 output data

This is an example pydelmod Notebook.

The data used are __hypothetical__. DSS files can be found in the release page. Place them under scenarios directory.

Create a Python virtual environment using 'environment.yaml' in the repository to install prerequisites.

In [ ]:
%load_ext autoreload

# Setting up

## Import modules
Import Pandas, plotly, and pydelmod. You might see some FutureWarnings.

In [ ]:
import pandas as pd
import plotly.offline as py
py.init_notebook_mode(connected=True)

import pydelmod.utilities as pdmu
import pydelmod.nbplot as pdmn

## Read in stations to review
A list of stations IDs and long names to review are read in.

In [ ]:
# Read output locations
fpath_output_locations = "scenarios/DSM2_output_locations_ec.csv"
df_stations = pd.read_csv(fpath_output_locations, comment='#')
df_stations['ID'] = [x.upper() for x in df_stations['ID']]
station_ids = df_stations['ID'].values

## Read post-processed DSM2 EC data in DSS

In [ ]:
# Read in data from DSS files
scenarios = [{'name': 'H3', 'fpath': "scenarios/H3/bdcp_h3_slr15_out_ec_PostPro.DSS"},
             {'name': 'H4', 'fpath': "scenarios/H4/bdcp_h4_slr15_out_ec_PostPro.DSS"}]
stations_to_read = df_stations['ID'].values

# Read only 'EC-MEAN'
variables_to_read = ['EC-MEAN', ]
intervals_to_read = None
dfs = []
for scenario in scenarios:
    fpath = scenario['fpath']
    name = scenario['name']
    df = pdmu.read_dss_to_df(fpath, bparts_to_read=stations_to_read,
                             cparts_to_read=variables_to_read, eparts_to_read=intervals_to_read)
    df['scenario_name'] = name
    dfs.append(df)

### Extract information from DSS pathnames and store into DataFrame columns
This could take some time depending on the size of the data.

In [ ]:
df_ec = pd.concat(dfs)
ds_parts = df_ec['pathname'].map(lambda x: (x.split('/')))
df_ec = df_ec.assign(station=ds_parts.map(lambda x: x[2]),
                     variable=ds_parts.map(lambda x: x[3]),
                     interval=ds_parts.map(lambda x: x[5]),
                     year=lambda x: x['time'].map(lambda y: y.year),
                     month=lambda x: x['time'].map(lambda y: y.month))

In [ ]:
# Add a wateryear type column
df_ec = df_ec.assign(wateryear=lambda x: x['year'])
mask = df_ec['month'] > 9
df_ec.loc[mask, 'wateryear'] += 1

## Read water year types from a CSV file
The CSV file is a dump from http://cdec.water.ca.gov/reportapp/javareports?name=WSIHIST.

In [ ]:
# Read water year types and join
df_wyt = pdmu.read_hist_wateryear_types('scenarios/wy_types.txt')
df_ec = df_ec.join(df_wyt.set_index('wy')['sac_yrtype'], on='wateryear')

## Choose the time period to review

In [ ]:
df_ec = df_ec[df_ec['time'] > '1974-12-31']

In [ ]:
# Create two dataframes for 1-Day and 1-MON interval data
df_1day = df_ec[df_ec['interval'] == '1DAY']
df_1mon = df_ec[df_ec['interval'] == '1MON']

# EC line plot

The plot below shows an interactive daily EC changes. You can pick a variable and station to review in the selection boxes below the plot.

- Placing a cursor on the curves will show values.
- You will see a toolbar at the top right corner of the plot when you move your cursor into the plot. You can zoom in and out, reset the zoom, export the plot, etc. by selecting one of the tools.
- You can zoom in and out by dragging the edges and corners of the plot as well.
- Clicking 'Show Data' button will show the actual data used for the plot in a tabular format. You can filter data to investigate the raw data. Click the button one more will hide the table.
- Clicking 'Save data' will export the data into 'export.csv' in the current directory where this script resides.
- Clicking 'Export Plots' will export plots of all the stations in the station list. The output format is PNG, and the name of the files will be the PREFIX_STATIONID.png using the PREFIX in the box next to the button. Depending on the number of stations and the size of the data, this could take some time. At the end of the exporting, it will generate a text file, PREFIX_description.csv, containing the list of the plot filenames and station names.

In [ ]:
pdmn.plot_step_w_variable_station_filters(df_1day, df_stations)

# Montly EC bar plots
The plot below shows an interactive monthly avergade EC values. It averages data monthly on the fly. You can pick a variable and station to review in the selection boxes below the plot. There are five wateryear type filter toggle buttons below the plot: Only selected wateryears by the buttons will be included for the plot.

- Placing a cursor on the curves will show values.
- You will see a toolbar at the top right corner of the plot when you move your cursor into the plot. You can zoom in and out, reset the zoom, export the plot, etc. by selecting one of the tools.
- Clicking 'Show Data' button will show the actual data used in the plot in a tabular format. You can filter data to investigate the raw data. Click the button one more will hide the table.
- Clicking 'Save data' will export the data into 'export.csv' in the current directory where this script resides.
- Clicking 'Export Plots' will export plots of all the stations in the station list. The output format is PNG, and the name of the files will be the PREFIX_STATIONID.png using the PREFIX in the box next to the button. Depending on the number of stations and the size of the data, this could take some time. At the end of the exporting, it will generate a text file, PREFIX_description.csv, containing the list of the plot filenames and station names.

In [ ]:
options = {'yaxis': 'EC (micromhos)', 'title': 'EC Means by Months', 'use_wateryear': True}
pdmn.plot_bar_monthly_w_controls(df_1day, df_stations, options)

# EC Exceedance curve
The plot below shows an interactive exceedance curve of EC. You can pick a variable and station to review in the selection boxes below the plot. There are five wateryear type filter toggle buttons below the plot: Only selected wateryears by the buttons will be included for the plot. There are month toggle buttons, which works the same ways as the wateryear type buttons.

- Placing a cursor on the curves will show values.
- You will see a toolbar at the top right corner of the plot when you move your cursor into the plot. You can zoom in and out, reset the zoom, export the plot, etc. by selecting one of the tools.
- Clicking 'Show Data' button will show the actual data used in the plot in a tabular format. You can filter data to investigate the raw data. Click the button one more will hide the table.
- Clicking 'Save data' will export the data into 'export.csv' in the current directory where this script resides.
- Clicking 'Export Plots' will export plots of all the stations in the station list. The output format is PNG, and the name of the files will be the PREFIX_STATIONID.png using the PREFIX in the box next to the button. Depending on the number of stations and the size of the data, this could take some time. At the end of the exporting, it will generate a text file, PREFIX_description.csv, containing the list of the plot filenames and station names.m

In [ ]:
pdmn.plot_exceedance_w_variable_station_filters(df_1day, df_stations)

# EC Boxplot
The plot below shows an interactive boxplots of EC. You can pick a variable and station to review in the selection boxes below the plot. There are five wateryear type filter toggle buttons below the plot: Only selected wateryears by the buttons will be included for the plot. There are month toggle buttons, which works the same ways as the wateryear type buttons.

- Placing a cursor on the curves will show values and statistics such as quantiles.
- You will see a toolbar at the top right corner of the plot when you move your cursor into the plot. You can zoom in and out, reset the zoom, export the plot, etc. by selecting one of the tools.
- Clicking 'Show Data' button will show the actual data used in the plot in a tabular format. You can filter data to investigate the raw data. Click the button one more will hide the table.
- Clicking 'Save data' will export the data into 'export.csv' in the current directory where this script resides.
- Clicking 'Export Plots' will export plots of all the stations in the station list. The output format is PNG, and the name of the files will be the PREFIX_STATIONID.png using the PREFIX in the box next to the button. Depending on the number of stations and the size of the data, this could take some time. At the end of the exporting, it will generate a text file, PREFIX_description.csv, containing the list of the plot filenames and station names.

In [ ]:
pdmn.plot_box_w_variable_station_filters(df_ec, df_stations)

# EC analysis with D1641 Ag standards

## Read in D1641 Agriculture standards
Following codes read in D1641 water quality objetives for agricultural beneficial uses: https://www.waterboards.ca.gov/waterrights/board_decisions/adopted_orders/decisions/d1600_d1649/wrd1641_1999dec29.pdf

In [ ]:
fpath_d1641_ag = './scenarios/D1641_AG_Standards.csv'
df_reg = pdmu.read_regulations(fpath_d1641_ag, df_wyt)
df_reg['value'] = df_reg['value'] * 1000.  # Convert milli to micro

In [ ]:
df_stations_reg = df_stations[df_stations['ID'].isin(df_reg['location'].unique())]

In [ ]:
# Get the time range from the data set
df_reg_ts = pdmu.generate_regulation_timeseries(df_reg, df, freq='D')
df_reg_ts['variable'] = 'EC-MEAN-14DAY'
regulation_name = 'D1641 AG'
df_reg_ts['scenario_name'] = 'D1641 AG'

In [ ]:
df = df_1day[df_1day['variable'] == 'EC-MEAN']

## EC time series with Ag standards
Similarly to the other plots above, you can choose stations, show and export values and plots, and zoom in and out the plot, etc.

In [ ]:
pdmn.plot_step_w_regulation(df, df_reg_ts, df_stations_reg)

## EC exceedance and statistics for D1641 Ag standards
The plot shows the exceedance of the EC difference from the D1641 Ag standards. The positive EC difference means that the EC goes over the standard at the station.

The plot also shows a table with statistics.

In [ ]:
pdmn.plot_exceedance_w_regulation(df, df_reg_ts, df_stations_reg)

# EC analysis with D1641 Fish and Wildlife standards

## Read in D1641 Fish and Wildlife standards
Following codes read in D1641 water quality objetives for fish and wildlife beneficial uses: https://www.waterboards.ca.gov/waterrights/board_decisions/adopted_orders/decisions/d1600_d1649/wrd1641_1999dec29.pdf

In [ ]:
# read in D1641 FWS standards
fpath_d1641_fws = './scenarios/D1641_FWS_Standards.csv'
df_reg_fws = pdmu.read_regulations(fpath_d1641_fws, df_wyt)
df_reg_fws['value'] = df_reg_fws['value'] * 1000.  # Convert milli to micro

In [ ]:
df_stations_reg_fws = df_stations[df_stations['ID'].isin(df_reg_fws['location'].unique())]

In [ ]:
# Get the time range from the data set
df_reg_fws_ts = pdmu.generate_regulation_timeseries(df_reg_fws, df, freq='D')
df_reg_fws_ts['variable'] = 'EC-MEAN-14DAY'
regulation_name = 'D1641 FWS'
df_reg_fws_ts['scenario_name'] = 'D1641 FWS'

## EC time series with Fish and Wildlife standards
Similarly to the other plots above, you can choose stations, show and export values and plots, and zoom in and out the plot, etc.

__NOTE: the deficiency year rule is not implemented yet.__

In [ ]:
pdmn.plot_step_w_regulation(df, df_reg_fws_ts, df_stations_reg_fws)

## EC exceedance and statistics for D1641 Fish and Wildlife standards
The plot shows the exceedance of the EC difference from the D1641 fish and wildlife standards. The positive EC difference means that the EC goes over the standard at the station.

The plot also shows a table with statistics.

__NOTE: the deficiency year rule is not implemented yet.__

In [ ]:
pdmn.plot_exceedance_w_regulation(df, df_reg_fws_ts, df_stations_reg_fws)